In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mne
import pandas as pd
import os
import matplotlib
import seaborn as sns
import scipy
from scipy.signal import find_peaks
from scipy.stats import mode
from itertools import compress

import pingouin as pg

from mpl_toolkits.axes_grid1 import make_axes_locatable, ImageGrid

# import pickle
# from alice_ml.models import predict_mne

# from biosppy.signals import ecg
# import pyhrv.tools as tools
# import pyhrv
# from hrvanalysis import remove_outliers, remove_ectopic_beats, interpolate_nan_values
# from hrvanalysis import get_time_domain_features

from scipy.stats import ttest_ind
from mne.channels import find_ch_adjacency, make_1020_channel_selections
from mne.stats import spatio_temporal_cluster_test

In [3]:
arr = np.load('/home/avaldamon/Desktop/intact/topomaps_np/AA_ME_smr_rest_topo.npy')
arr.shape

(204,)

In [2]:
fnirs_data_folder = '/mnt/diskus/fNIRS data ME_MI_TS_TI_SA/OK/OK_TS'
raw_intensity = mne.io.read_raw_nirx(fnirs_data_folder, verbose=True)

Loading /mnt/diskus/fNIRS data ME_MI_TS_TI_SA/OK/OK_TS


In [19]:
events, event_dict = mne.events_from_annotations(raw_intensity)


Used Annotations descriptions: ['1.0', '2.0']


In [31]:
events_sequence = events[:, [2]].flatten()
events_sequence

array([2, 2, 1, 1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2,
       2, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2])

In [16]:
raw_intensity.times.shape

(7214,)

In [6]:
wings_file = '/mnt/diskus/fNIRS data ME_MI_TS_TI_SA/OK/OK_TS/2023-07-17_001.wings'
df_wings = pd.read_csv(wings_file, sep=';',
               header=None)

In [14]:
%matplotlib auto
sns.lineplot(df_wings[5])

Using matplotlib backend: QtAgg


<AxesSubplot: ylabel='5'>

In [18]:
for i in range(1,13,1):
    plt.figure()
    plt.plot(df_wings[i])
    plt.xlim(30000,50000)
    plt.title(i)

In [21]:
eventsInms = [raw_intensity.times[i] for i in events[:,0]]
winxTimes = a[1].to_numpy()
winxTimes_events = [np.where(winxTimes<i)[0][-1] for i in eventsInms]

In [40]:
wings_events = np.array(winxTimes_events)
wings_events = np.vstack((wings_events, events_sequence))

In [42]:
wings_events

array([[ 46444,  54064,  61682,  69303,  76919,  84538,  92156,  99774,
        107393, 115010, 122630, 130249, 137867, 145436, 153055, 160672,
        168292, 175910, 183580, 191150, 198815, 206433, 214002, 221621,
        229239, 236856, 244475, 252097, 259714, 267334, 274905, 282519,
        290187, 297758, 305380, 312991, 320613, 328183, 335801, 343468],
       [     2,      2,      1,      1,      2,      1,      1,      2,
             2,      1,      1,      2,      2,      1,      1,      2,
             1,      1,      2,      1,      1,      2,      2,      1,
             1,      2,      2,      1,      2,      2,      1,      1,
             2,      2,      1,      2,      2,      1,      1,      2]])

In [43]:
for e in wings_events:
    print(e)

[ 46444  54064  61682  69303  76919  84538  92156  99774 107393 115010
 122630 130249 137867 145436 153055 160672 168292 175910 183580 191150
 198815 206433 214002 221621 229239 236856 244475 252097 259714 267334
 274905 282519 290187 297758 305380 312991 320613 328183 335801 343468]
[2 2 1 1 2 1 1 2 2 1 1 2 2 1 1 2 1 1 2 1 1 2 2 1 1 2 2 1 2 2 1 1 2 2 1 2 2
 1 1 2]


In [57]:
from scipy.signal import detrend

signal = detrend(df_wings[5]) 

plt.plot(signal)
for epochs in range(wings_events.shape[1]):
    epoch_time = wings_events[0, epochs]
    epoch_type = wings_events[1, epochs]
    
    print(epoch_type)
    if epoch_type == 2:
        plt.axvline(epoch_time, color='r')
    elif epoch_type == 1:
        plt.axvline(epoch_time, color='b')


plt.xlim(1000,3e5)

2
2
1
1
2
1
1
2
2
1
1
2
2
1
1
2
1
1
2
1
1
2
2
1
1
2
2
1
2
2
1
1
2
2
1
2
2
1
1
2


(1000.0, 300000.0)